In [1]:
#settings 
from settings.settings import data_settings 
from settings.settings import model_settings

#imports
import pandas as pd 
import numpy as np 
# import tensorflow as tf
# import keras
# from keras import layers
# import pathlib
# import datetime
# from model.blocks.keras_tcn import TCN

In [2]:
# Loading data and making it ready

train_df = pd.read_csv(data_settings.data_train_path)

test_df = pd.read_csv(data_settings.data_test_path)

test_buy = test_df.pop(test_df.buy_4.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.sell_4.name)

train_buy = train_df.pop(train_df.buy_4.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.sell_4.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)


In [5]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [55]:
# y_train = pd.DataFrame([train_buy,train_nothing,train_sell]).transpose()
# y_test = pd.DataFrame([test_buy,test_nothing,test_sell]).transpose()


X_train = (train_df - train_df.mean())/train_df.std()
X_test = (test_df - train_df.mean())/train_df.std()

y_train = []
for buy,nothing,sell in zip(train_buy,train_nothing,train_sell):
    if(buy == 1):
        y_train.append(0)
    elif(nothing == 1):
        y_train.append(1)
    elif(sell == 1):
        y_train.append(2)

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
# print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)


In [13]:
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score

size = 10_000
max_depth=8
learning_rate = 0.1
max_bin=1024

model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        max_bin=max_bin,
                        grow_policy='depthwise',
                        learning_rate=learning_rate,
                        verbosity = 3,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        missing=np.nan,
                        num_parallel_tree=size,
                        monotone_constraints='()',
                        interaction_constraints='',
                        importance_type=None,
                        gpu_id=0,
                        predictor='auto',
                        enable_categorical=False,
                        # max_cat_to_onehot=4,
                        # eval_metric = log_loss,
                        early_stopping_rounds=None,
                        callbacks=None,
                        max_delta_step=0,
                        num_class = 3,
                        objective='multi:softmax',
                        )

In [14]:
# eval_set = [(X_test, y_test)]

model.fit(
            X_train,
            y_train,
            # eval_set = eval_set,
            verbose = 32
        )


[07:16:02] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_gpu_hist.cu:817: [GPU Hist]: Configure
[07:16:02] ======== Monitor (0): SketchContainer ========
[07:16:02] MakeCuts: 0.00243s, 1 calls @ 2430us

[07:16:02] Prune: 0.001324s, 1 calls @ 1324us

[07:16:02] ScanInput: 0.013069s, 1 calls @ 13069us

[07:16:02] Unique: 0.000905s, 1 calls @ 905us

[08:34:56] ======== Monitor (0): Learner ========
[08:34:56] Configure: 0.001701s, 1 calls @ 1701us

[08:34:56] EvalOneIter: 7.9e-05s, 1 calls @ 79us

[08:34:56] GetGradient: 0.00648s, 1 calls @ 6480us

[08:34:56] PredictRaw: 0.198273s, 1 calls @ 198273us

[08:34:56] UpdateOneIter: 4729.89s, 1 calls @ 4729885622us

[08:34:56] ======== Monitor (0): GBTree ========
[08:34:56] BoostNewTrees: 4729.68s, 1 calls @ 4729675890us

[08:34:56] CommitModel: 0.002854s, 1 calls @ 2854us

[08:34:56] ======== Device 0 Memory Allocations:  ========
[08:34:56] Peak memory usage: 591MiB
[08:34:56] Number of allocations: 107

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints='',
                learning_rate=0.1, max_bin=1024, max_cat_to_onehot=4,
                max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=1,
                missing=nan, monotone_constraints='()', n_estimators=10000,
                n_jobs=0, num_class=3, num_parallel_tree=10000,
                objective='multi:softmax', predictor='auto', random_state=23,
                reg_alpha=0, ...)

In [15]:
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
import pathlib

acc_train = accuracy_score(p_train,y_train)
acc_test = accuracy_score(p_test,y_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)

con_train = confusion_matrix(y_train,p_train)
con_test = confusion_matrix(y_test,p_test)


#check if path is available
path = f'./model/models/XGBOOST/model-{size}-{max_depth}-{learning_rate}-{max_bin}'
pathlib.Path(path).mkdir(parents=True,exist_ok=True)
model.save_model(f"{path}/model.json")

with open(f"{path}/train_report.txt","w") as f:
    f.write(rep_train)

with open(f"{path}/test_report.txt","w") as f:
    f.write(rep_test) 
    
print(rep_train)
print(rep_test)


              precision    recall  f1-score   support

           0       0.97      0.08      0.15     44559
           1       0.70      1.00      0.82    189479
           2       0.95      0.06      0.11     44666

    accuracy                           0.70    278704
   macro avg       0.87      0.38      0.36    278704
weighted avg       0.78      0.70      0.60    278704

              precision    recall  f1-score   support

           0       0.21      0.05      0.08      6886
           1       0.68      0.94      0.79     33141
           2       0.19      0.04      0.06      9157

    accuracy                           0.64     49184
   macro avg       0.36      0.34      0.31     49184
weighted avg       0.52      0.64      0.55     49184





1000 , 10
              precision    recall  f1-score   support

           0       0.98      0.16      0.28     44559
           1       0.71      1.00      0.83    189479
           2       0.98      0.11      0.20     44666

    accuracy                           0.72    278704
   macro avg       0.89      0.42      0.44    278704
weighted avg       0.80      0.72      0.64    278704

              precision    recall  f1-score   support

           0       0.17      0.34      0.23      6886
           1       0.70      0.67      0.68     33141
           2       0.18      0.08      0.11      9157

    accuracy                           0.51     49184
   macro avg       0.35      0.36      0.34     49184
weighted avg       0.53      0.51      0.51     49184

BASE
              precision    recall  f1-score   support

           0       0.89      0.05      0.09     44559
           1       0.69      0.99      0.81    189479
           2       0.89      0.03      0.06     44666

   micro avg       0.69      0.69      0.69    278704
   macro avg       0.82      0.36      0.32    278704
weighted avg       0.75      0.69      0.58    278704
 samples avg       0.68      0.69      0.69    278704

              precision    recall  f1-score   support

           0       0.07      0.01      0.01      6886
           1       0.69      0.91      0.78     33141
           2       0.27      0.00      0.01      9157

   micro avg       0.68      0.62      0.65     49184
   macro avg       0.34      0.31      0.27     49184
weighted avg       0.52      0.62      0.53     49184
 samples avg       0.61      0.62      0.61     49184


In [ ]:
# from xgboost import plot_importance ,plot_tree
# import matplotlib as mpl

# # model = XGBRFClassifier()
# # model.load_model(f"./model/models/XGBOOST/model_1000_12/model.json")

# mpl.rcParams['axes.grid'] = False
# mpl.rcParams['figure.figsize'] = (200, 100)

# plot_tree(model)


In [56]:
df = X_test.copy()
df = df * train_df.std() + train_df.mean()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))
df

,Open,High,Low,Close,TickVol,Spread,ATR,Open_P,High_P,Low_P,...,EMA_INDICATOR_100,SMA_INDICATOR_12,SMA_INDICATOR_26,SMA_INDICATOR_50,SMA_INDICATOR_100,Day_sin,Day_cos,Year_sin,Year_cos,DateTime
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-09-29 19:10:00,1.16121,1.16134,1.16104,1.16117,253.0,0.0,0.000423,0.999819,0.999923,0.999931,...,1.164043,1.161318,1.161571,1.162970,1.164483,0.023142,0.650353,0.000167,0.487093,2021-09-29 19:10:00
2021-09-29 19:15:00,1.16118,1.16127,1.16103,1.16103,202.0,0.0,0.000409,0.999974,0.999940,0.999991,...,1.163984,1.161303,1.161489,1.162884,1.164431,0.026535,0.660720,0.000166,0.487123,2021-09-29 19:15:00
2021-09-29 19:20:00,1.16101,1.16103,1.16085,1.16089,178.0,0.0,0.000393,0.999854,0.999793,0.999845,...,1.163922,1.161238,1.161428,1.162796,1.164375,0.030154,0.671010,0.000165,0.487152,2021-09-29 19:20:00
2021-09-29 19:25:00,1.16088,1.16114,1.16080,1.16095,208.0,0.0,0.000389,0.999888,1.000095,0.999957,...,1.163863,1.161176,1.161385,1.162703,1.164320,0.033996,0.681219,0.000164,0.487182,2021-09-29 19:25:00
2021-09-29 19:30:00,1.16095,1.16105,1.16058,1.16062,210.0,0.0,0.000395,1.000060,0.999922,0.999810,...,1.163799,1.161098,1.161341,1.162600,1.164261,0.038060,0.691342,0.000164,0.487212,2021-09-29 19:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-26 23:20:00,1.07281,1.07288,1.07277,1.07279,98.0,4.0,0.000266,1.000009,1.000028,0.999991,...,1.071519,1.072491,1.072275,1.071817,1.071493,0.413176,0.992404,0.792044,0.094155,2022-05-26 23:20:00
2022-05-26 23:25:00,1.07279,1.07302,1.07278,1.07291,104.0,4.0,0.000264,0.999981,1.000130,1.000009,...,1.071547,1.072570,1.072315,1.071827,1.071520,0.423938,0.994181,0.792020,0.094137,2022-05-26 23:25:00
2022-05-26 23:30:00,1.07291,1.07307,1.07286,1.07301,127.0,8.0,0.000260,1.000112,1.000047,1.000075,...,1.071576,1.072617,1.072348,1.071848,1.071554,0.434737,0.995722,0.791995,0.094120,2022-05-26 23:30:00


In [57]:
df["Signal"] = p_test
df

,Open,High,Low,Close,TickVol,Spread,ATR,Open_P,High_P,Low_P,...,SMA_INDICATOR_12,SMA_INDICATOR_26,SMA_INDICATOR_50,SMA_INDICATOR_100,Day_sin,Day_cos,Year_sin,Year_cos,DateTime,Signal
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-09-29 19:10:00,1.16121,1.16134,1.16104,1.16117,253.0,0.0,0.000423,0.999819,0.999923,0.999931,...,1.161318,1.161571,1.162970,1.164483,0.023142,0.650353,0.000167,0.487093,2021-09-29 19:10:00,1
2021-09-29 19:15:00,1.16118,1.16127,1.16103,1.16103,202.0,0.0,0.000409,0.999974,0.999940,0.999991,...,1.161303,1.161489,1.162884,1.164431,0.026535,0.660720,0.000166,0.487123,2021-09-29 19:15:00,1
2021-09-29 19:20:00,1.16101,1.16103,1.16085,1.16089,178.0,0.0,0.000393,0.999854,0.999793,0.999845,...,1.161238,1.161428,1.162796,1.164375,0.030154,0.671010,0.000165,0.487152,2021-09-29 19:20:00,1
2021-09-29 19:25:00,1.16088,1.16114,1.16080,1.16095,208.0,0.0,0.000389,0.999888,1.000095,0.999957,...,1.161176,1.161385,1.162703,1.164320,0.033996,0.681219,0.000164,0.487182,2021-09-29 19:25:00,1
2021-09-29 19:30:00,1.16095,1.16105,1.16058,1.16062,210.0,0.0,0.000395,1.000060,0.999922,0.999810,...,1.161098,1.161341,1.162600,1.164261,0.038060,0.691342,0.000164,0.487212,2021-09-29 19:30:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-26 23:20:00,1.07281,1.07288,1.07277,1.07279,98.0,4.0,0.000266,1.000009,1.000028,0.999991,...,1.072491,1.072275,1.071817,1.071493,0.413176,0.992404,0.792044,0.094155,2022-05-26 23:20:00,0
2022-05-26 23:25:00,1.07279,1.07302,1.07278,1.07291,104.0,4.0,0.000264,0.999981,1.000130,1.000009,...,1.072570,1.072315,1.071827,1.071520,0.423938,0.994181,0.792020,0.094137,2022-05-26 23:25:00,0
2022-05-26 23:30:00,1.07291,1.07307,1.07286,1.07301,127.0,8.0,0.000260,1.000112,1.000047,1.000075,...,1.072617,1.072348,1.071848,1.071554,0.434737,0.995722,0.791995,0.094120,2022-05-26 23:30:00,0


In [78]:
from backtesting import Strategy

def Signal():
    return df.Signal

class MyStrategy(Strategy):
    initsize = 0.05
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()
        if(self.signal1 != 1 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
            self.mysize=self.mysize*2
            
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            self.mysize=self.initsize
            
        if self.signal1==0:
            sl1 = self.data.Close[-1] -self.data.ATR[-1] * 3
            tp1 = self.data.Close[-1] + 12* self.data.ATR[-1]
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        elif self.signal1==2:
            sl1 = self.data.Close[-1] + (self.data.ATR[-1]*3)
            tp1 = self.data.Close[-1] -( 12 * self.data.ATR[-1])
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

In [80]:
from backtesting import Backtest

bt = Backtest(df, MyStrategy, cash=1_000, margin=1/10, commission=.0002)
stat = bt.run()
stat



AssertionError: size must be a positive fraction of equity, or a positive whole number of units

In [ ]:
# Creating generators

# window_size = data_settings.window_size
# batch_size : int = 256
# num_features : int = 97
# num_classes : int = 3

# def x_train_generator():
#     for i in range(window_size,len(train_df)+1):
#         window=train_df[i-window_size:i]
#         yield window.to_numpy()

# def x_test_generator():
#     for i in range(window_size,len(test_df)+1):
#         window=test_df[i-window_size:i]
#         yield window.to_numpy()    
        
# def y_train_generator():
#     for i in range(window_size,train_labels.shape[0]+1):
#         yield train_labels[i-1]

# def y_test_generator():
#     for i in range(window_size,test_labels.shape[0]+1):
#         yield test_labels[i-1]
        
# x_train_ds = tf.data.Dataset.from_generator(x_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)

# x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)
# y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)
# y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)

# train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
# test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
# def compile_and_fit(model:keras.Model,modelname,train_ds=train_ds,test_ds=test_ds,modelcheckpoint:bool = False,epochs= 10):
#     import os

#     #check if path is available
#     path = f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit'
#     pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#     #tensorboard
#     log_dir =f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     #modelcheckpoint
#     base_path = f"./model/models/{modelname}/"
#     checkpoint_path = base_path + "tmp/cp-{epoch:04d}.ckpt"
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_path,
#         monitor='val_loss',
#         verbose=1,
#         save_weights_only=True,
#         save_best_only=False
#     )

#     model.save_weights(checkpoint_path.format(epoch=0))


#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=["categorical_accuracy","categorical_crossentropy"],
#                 )

#     if modelcheckpoint:
#         history = model.fit(
#                             train_ds,
#                             validation_data= test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback,model_checkpoint_callback],
#                             use_multiprocessing=True,
#                             )
#     else:
#         history = model.fit(    
#                             train_ds,
#                             validation_data=test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback],
#                             use_multiprocessing=True,
#                            )
    
#     return history

In [ ]:
#TF Model
# input_layer = layers.Input(shape=(window_size,num_features))
# x = TCN(
#     nb_filters=128,
#     # kernel_size=3,
#     # nb_stacks=1,
#     # padding="causal",
#     # activation="swish",
#     use_batch_norm=True,
#     # return_sequences=True
#     )(input_layer)

# # x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )

# input_layer = layers.Input(shape=(window_size,num_features))

# x = layers.Dense(1024,activation="relu")(input_layer)
# x = layers.Dense(512,activation="relu")(x)
# x = layers.Dense(256,activation="relu")(x)
# x = layers.Dense(128,activation="relu")(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.Dense(32,activation="relu")(x)
# x = layers.Dense(16,activation="relu")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )
# model.summary()
# compile_and_fit(
#     model=model,
#     modelname="simple_dense_first_try",
#     train_ds=train_ds,
#     test_ds=test_ds,
#     modelcheckpoint=True
# )